In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from itertools import chain
from sys import platform
from copy import deepcopy as dc
from scipy.interpolate import interp1d
from astropy.time import Time

vims_wave = np.loadtxt('vims_wave.txt')

new_array = ['_0405','_0607','_0809','_1011','_1213','_1415','_1617']
if platform == 'darwin' or platform == 'win32' or platform == 'linux':
    for number in new_array:
        new_string = 'df'+number+' = pd.read_pickle("total_data/data'+number+'.pkl")'
        exec(new_string)
else:
    df_list = []
    for number in new_array:
        new_string = 'df'+number+' = pd.read_csv("total_data/data'+number+'.csv")'
        exec(new_string)
        df_list.append(locals()['df'+number])
    for df in df_list:
        for i in range(len(df['spectrum'])):
            df['spectrum'][i] = np.array([float(x) for x in df['spectrum'][i][1:-1].split(',')])

vims_list = []
for i in range(4,18):
    if i<10:
        exec('vims_200'+str(i)+' = np.load("vims_shift/vims_200'+str(i)+'.npy")')
        vims_list.append(locals()['vims_200'+str(i)])
    else:
        exec('vims_20'+str(i)+' = np.load("vims_shift/vims_20'+str(i)+'.npy")')
        vims_list.append(locals()['vims_20'+str(i)])

def powerlaw(x,a,b):
    return a*np.power(x,b)

def gaussian(x,a,mu,sigma):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))

def flatten(some_list):
    flat_list = []
    for element in some_list:
        if type(element) is list or type(element) is tuple:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

band_channels = list(chain(range(29,35),range(46,60),range(78,96),range(102,106)))

window = list(range(56,83))

def fit_line(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_fit = powerlaw(vims_wave,*my_fit)
    return final_fit

def custom_fit(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_spectra = spectra-powerlaw(vims_wave,*my_fit)
    return final_spectra

def single_fit(spectrum):
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectrum[band_channels],p0=[.15,-12])
    final_spectrum = spectrum-powerlaw(vims_wave,*my_fit)
    return final_spectrum

def shift_ret(spectrum):
    my_fit,_ = curve_fit(gaussian,vims_wave[window],spectrum[window],p0=[.05,2.05])
    return my_fit[1]

In [2]:
scatter_list = []
for number in new_array:
    my_string = 'scatter'+number+" = df"+number
    exec(my_string)
    scatter_list.append(locals()['scatter'+number])

airmass_data = pd.concat(scatter_list,ignore_index=True)

all_spectra = airmass_data['spectrum']

airmass_data['emission'] *= (np.pi)/180
airmass_data['incidence'] *= (np.pi)/180

spec_list = []
for number in new_array:
    my_string = 'old_spec'+number+' = df'+number+"['spectrum']"
    exec(my_string)
    new_string = 'spec'+number+' = np.array([np.abs(x) for x in old_spec'+number+'])'
    exec(new_string)
    spec = 'spec'+number
    spec_list.append(locals()[spec])

averaged_list = []
for number in new_array:
    my_string = 'averaged'+number+' = np.average(spec'+number+',axis=0)'
    exec(my_string)
    averaged_list.append(locals()['averaged'+number])

copy_list = []
for number in new_array:
    my_string = 'copy_spec'+number+' = dc(spec'+number+')'
    exec(my_string)
    copy_spec = 'copy_spec'+number
    copy_list.append(locals()[copy_spec])

copy_list_total = []
for (spec,copy_spec) in zip(spec_list,copy_list):
    for i in range(256):
        filtered = False
        k = 0
        while filtered == False:
            mean_1 = np.mean(copy_spec[:,i])
            median = np.median(copy_spec[:,i])
            std = np.std(copy_spec[:,i])
            vals = copy_spec[:,i]
            ind = np.where(np.abs(vals-mean_1)>2*std)
            copy_spec[ind,i] = median
            mean_2 = np.mean(copy_spec[:,i])
            k += 1
            if np.abs(mean_1-mean_2) <=.05*mean_1:
                filtered = True
    for spectrum in copy_spec:
        copy_list_total.append(spectrum)

channels = [
(9,20,37,65,107),
(12,24,43,70,115),
(16,27,47,76,118)
]

k_vals = [1.14,1.29,1.60,1.50,1.15]

channels = np.asarray(channels)
    
airmass_data['spectrum'] = copy_list_total

i_data = dc(airmass_data)
k_data = dc(airmass_data)

values = []
for spectrum in airmass_data['spectrum']:
    values.append(spectrum[channels[:,1]])
values = np.asarray(values)

In [3]:
all_spectra = []
for i in range(len(airmass_data['spectrum'])):
    all_spectra.append(airmass_data['spectrum'][i])
all_spectra = np.asarray(all_spectra)

In [6]:
i_data['spectrum'] /= np.cos(i_data['incidence'])

airmass_data['spectrum'] = (airmass_data['spectrum']*np.cos(airmass_data['incidence']))/((np.cos(airmass_data['incidence']))+(np.cos(airmass_data['emission'])))

corrected_spectra = dc(all_spectra)
for i in range(5):
    width = 1
    cor = (k_vals[i]/2)*(all_spectra[:,channels[0,i]]+all_spectra[:,channels[2,i]])
    corrected_spectra[:,channels[0,i]+width:channels[2,i]-width] -= cor[:,None]

test_cos = dc(corrected_spectra)
inc = np.cos(airmass_data['incidence'])[:,None]
emi = np.cos(airmass_data['emission'])[:,None]
test_cos = (test_cos*inc)/(inc+emi)

/var/folders/p8/y355zbcj5c5343tcpnn7rh2r0000gn/T/ipykernel_11461/1503316402.py:12: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  inc = np.cos(airmass_data['incidence'])[:,None]
/var/folders/p8/y355zbcj5c5343tcpnn7rh2r0000gn/T/ipykernel_11461/1503316402.py:13: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  emi = np.cos(airmass_data['emission'])[:,None]


In [7]:
shift_dates = [
'2004-01-01T00:00:00.00',
'2005-01-01T00:00:00.00',
'2005-07-01T00:00:00.00',
'2006-01-01T00:00:00.00',
'2006-07-01T00:00:00.00',
'2007-01-01T00:00:00.00',
'2007-07-01T00:00:00.00',
'2008-01-01T00:00:00.00',
'2008-07-01T00:00:00.00',
'2009-01-01T00:00:00.00',
'2009-07-01T00:00:00.00',
'2010-01-01T00:00:00.00',
'2010-07-01T00:00:00.00',
'2011-01-01T00:00:00.00',
'2011-07-01T00:00:00.00',
'2012-01-01T00:00:00.00',
'2012-07-01T00:00:00.00',
'2013-01-01T00:00:00.00',
'2013-07-01T00:00:00.00',
'2014-01-01T00:00:00.00',
'2014-04-19T00:00:00.00',
'2014-07-01T00:00:00.00',
'2015-01-01T00:00:00.00',
'2015-07-01T00:00:00.00',
'2016-01-01T00:00:00.00',
'2016-07-01T00:00:00.00',
'2017-01-01T00:00:00.00',
'2017-07-01T00:00:00.00',
'2017-10-19T00:00:00.00',
]

shift_vals = [0.0,0.0,0.4,1.1,1.9,2.6,3.4,4.1,4.9,5.6,6.0,6.0,6.0,6.0,7.0,7.3,7.7,8.0,8.4,8.7,9.1,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8]
shift_vals = np.asarray(shift_vals)
other_shifts = shift_vals/1000

date_vector = Time(shift_dates,format='isot').jd
date_interp = interp1d(date_vector,shift_vals)

all_shifts = date_interp(list(airmass_data['starttime']))
all_shifts = all_shifts/1000

In [8]:
k_avg = []
k_std = []
wave_shifts = [] # Works for *all* averages, not just one
for year in range(2004,2018):
    start_date = Time(str(year)+'-01-01T00:00:00.00',format='isot').jd
    end_date = Time(str(year+1)+'-01-01T00:00:00.00',format='isot').jd
    ind = np.where((airmass_data['starttime'] >= start_date) & (airmass_data['starttime'] <= end_date))[0]
    if len(ind) != 0:
        shift_ind = np.where(date_vector == Time(str(year)+'-07-01T00:00:00.00',format='isot').jd)[0][0]
        left_ind = np.where(date_vector == Time(str(year)+'-01-01T00:00:00.00',format='isot').jd)[0][0]
        right_ind = np.where(date_vector == Time(str(year+1)+'-01-01T00:00:00.00',format='isot').jd)[0][0]
        a = np.average(corrected_spectra[ind],axis=0)
        s = np.std(corrected_spectra[ind])
        k_avg.append(a)
        k_std.append(s)
        wave_shifts.append([other_shifts[left_ind],other_shifts[shift_ind],other_shifts[right_ind]])

wave_shifts = np.asarray(wave_shifts)

In [11]:
%matplotlib qt
fig,axs = plt.subplots(1,2,figsize=(24,8),facecolor='white',sharex=True,sharey=True)
year = 2012
start_date = Time(str(year)+'-01-01T00:00:00.00',format='isot').jd
end_date = Time(str(year+1)+'-01-01T00:00:00.00',format='isot').jd
inds = np.where((airmass_data['starttime'] >= start_date) & (airmass_data['starttime'] <= end_date))[0]
for ind in inds:
    axs[0].plot(vims_wave,corrected_spectra[ind])
    axs[1].plot(vims_wave,test_cos[ind])
plt.grid()
plt.show()

In [ ]:
%matplotlib inline
plt.figure(facecolor='white',figsize=(12,8))
for i in range(len(k_avg)):
    x = vims_wave+wave_shifts[i,1]
    y = k_avg[i]
    l_err = [wave_shifts[i,1]-wave_shifts[i,0]]*256
    r_err = [wave_shifts[i,2]-wave_shifts[i,1]]*256
    plt.plot(x,y)
    plt.errorbar(x,y,xerr=[l_err,r_err],yerr=k_std[i],fmt='.')
plt.grid()
plt.show()

In [10]:
%matplotlib qt
year = 2012
start_date = Time(str(year)+'-01-01T00:00:00.00',format='isot').jd
end_date = Time(str(year+1)+'-01-01T00:00:00.00',format='isot').jd
inds = np.where((airmass_data['starttime'] >= start_date) & (airmass_data['starttime'] <= end_date))[0]
for ind in inds:
    plt.plot(vims_wave,corrected_spectra[ind])
plt.grid()
plt.show()